# import libraries and data

In [1]:
from features import Dataframe
from utils import *
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import pandas as pd
import numpy as np

## create features and targets

In [27]:
df = Dataframe().get_features()

In [28]:
X = df.drop(columns=['predict', 'date', 'Value_classification'])
y = df['predict']

In [29]:
df

,date,current_price,current_price_sats,market_cap,reddit_post_48h,reddit_comment_48h,reddit_subscribers,reddit_active_accounts,public_interest_stats,Value,...,sats_change_2_weeks,price_change_2_days,price_change_1_week,price_change_2_weeks,percent_change_2_days,percent_change_1_week,percent_change_2_weeks,percent_sats_2_days,percent_sats_1_week,percent_sats_2_weeks
0,2020-08-19,2.936665,25024.566687,0.000000e+00,0.000,0.750,193,7.200000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-08-20,2.936665,25024.566687,0.000000e+00,0.000,0.667,212,5.500000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-08-21,2.898791,24435.409268,0.000000e+00,0.000,1.727,237,6.333333,NaN,NaN,...,NaN,-0.037874,NaN,NaN,-0.012897,NaN,NaN,-0.023543,NaN,NaN
3,2020-08-22,2.871923,24934.417517,0.000000e+00,0.417,2.750,269,8.538462,NaN,NaN,...,NaN,-0.064742,NaN,NaN,-0.022046,NaN,NaN,-0.003602,NaN,NaN
4,2020-08-23,4.481389,38367.856477,4.028277e+09,1.091,5.364,313,16.500000,NaN,NaN,...,NaN,1.582598,NaN,NaN,0.545951,NaN,NaN,0.570174,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,2022-05-08,13.800377,38810.570189,1.534235e+10,0.846,6.154,39879,59.857143,38454.0,18.0,...,-8614.010277,-0.798469,-0.769511,-4.940943,-0.054694,-0.052815,-0.263639,-0.026538,0.005946,-0.181636
628,2022-05-09,13.229361,38851.622851,1.469313e+10,0.600,5.100,39889,60.363636,38454.0,11.0,...,-7147.612935,-1.126357,-2.189413,-4.931293,-0.078460,-0.141997,-0.271537,-0.022777,-0.030164,-0.155385
629,2022-05-10,10.801430,35649.269219,1.205226e+10,2.000,16.556,39906,70.600000,38454.0,10.0,...,-9119.805926,-2.998947,-4.191323,-7.326671,-0.217309,-0.279557,-0.404161,-0.081455,-0.082992,-0.203708
630,2022-05-11,11.325015,36527.298076,1.252311e+10,0.917,11.500,39925,78.923077,38454.0,12.0,...,-7019.894754,-1.904346,-3.402347,-5.285691,-0.143948,-0.231022,-0.318210,-0.059826,-0.063454,-0.161202


## create x data that has no target

In [30]:
new_X = Dataframe().get_x_to_predict()

In [31]:
new_X

,date,current_price,current_price_sats,market_cap,reddit_post_48h,reddit_comment_48h,reddit_subscribers,reddit_active_accounts,public_interest_stats,Value,...,sats_change_2_weeks,price_change_2_days,price_change_1_week,price_change_2_weeks,percent_change_2_days,percent_change_1_week,percent_change_2_weeks,percent_sats_2_days,percent_sats_1_week,percent_sats_2_weeks
632,2022-05-13,8.769325,30114.350218,9.672795e+09,1.222,12.778,39965,78.900000,38454.0,10.0,...,-12617.781835,-2.555689,-5.829521,-8.213629,-0.225668,-0.399314,-0.483640,-0.175566,-0.244660,-0.295276
633,2022-05-14,10.487976,35798.763057,1.154770e+10,1.444,15.333,39974,73.600000,38454.0,9.0,...,-6053.638262,1.586772,-3.867741,-5.686264,0.178265,-0.269422,-0.351563,0.161123,-0.099565,-0.144643
634,2022-05-15,11.376092,37684.247686,1.273195e+10,1.250,15.250,39986,57.555556,38454.0,10.0,...,-896.934621,2.606767,-2.424285,-3.193796,0.297260,-0.175668,-0.219205,0.251372,-0.029021,-0.023248
635,2022-05-16,11.800641,37710.067770,1.308825e+10,1.222,19.000,39992,60.300000,51803.0,14.0,...,-2349.938512,1.312664,-1.428720,-3.618133,0.125159,-0.107996,-0.234658,0.053390,-0.029382,-0.058660
636,2022-05-17,10.661735,35619.793504,1.187424e+10,1.200,8.600,40003,64.363636,51803.0,8.0,...,-3255.834410,-0.714357,-0.139695,-4.331018,-0.062795,-0.012933,-0.288874,-0.054783,-0.000827,-0.083750
637,2022-05-18,11.053819,36248.812004,1.231961e+10,1.167,9.917,40019,66.153846,51803.0,12.0,...,-2753.331822,-0.746821,-0.271195,-3.673542,-0.063287,-0.023947,-0.249437,-0.038750,-0.007624,-0.070594
638,2022-05-19,9.517212,33098.858929,1.064163e+10,0.909,12.909,40037,64.333333,51803.0,13.0,...,-7944.990134,-1.144522,0.616008,-6.775156,-0.107349,0.069205,-0.415848,-0.070773,0.073552,-0.193573
639,2022-05-20,10.071635,33179.465129,1.121794e+10,2.500,20.125,40046,68.666667,51803.0,13.0,...,-6689.134681,-0.982184,1.302310,-4.527211,-0.088855,0.148507,-0.310107,-0.084674,0.101783,-0.167780
640,2022-05-21,9.695758,33118.399106,1.080997e+10,1.500,19.125,40049,60.222222,51803.0,13.0,...,-6638.784343,0.178546,-0.792218,-4.659959,0.018760,-0.075536,-0.324606,0.000590,-0.074873,-0.166983
641,2022-05-22,9.996885,33909.441680,1.115733e+10,1.111,11.222,40142,69.800000,51803.0,14.0,...,-4901.128508,-0.074750,-1.379207,-3.803491,-0.007422,-0.121237,-0.275608,0.022001,-0.100169,-0.126283


# create Pipelines

## determine initial value for k in SelectKBest

In [32]:
scaling_pipeline = Pipeline([
    ('imputer', SimpleImputer()),
    ('scaler', RobustScaler())])

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
Xk_train = scaling_pipeline.fit_transform(X_train)

In [34]:
grad_k, grad_score = Utils().find_best_features_gradient(Xk_train, y_train)
rand_k, rand_score = Utils().find_best_features_r_forest(Xk_train, y_train)

In [35]:
initial_grad_k = grad_k[np.argmax(grad_score)]
initial_rand_k = rand_k[np.argmax(rand_score)]

In [36]:
display(f"The initial value for k in the Gradient Boosting pipeline will be: {initial_grad_k}")
display(f"The initial value for k in the Random Forest pipeline will be: {initial_rand_k}")

'The initial value for k in the Gradient Boosting pipeline will be: 19'

'The initial value for k in the Random Forest pipeline will be: 19'

## create pipeline for Gradient Boosting Classifier

In [52]:
gradient_pipe = Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),
                ('scaler', RobustScaler()), 
                ('feature_select', SelectKBest(score_func=mutual_info_classif, k=19)),
                ('clf_grad', GradientBoostingClassifier(learning_rate=0.3, loss='log_loss',
                                                       max_depth=20, min_samples_leaf=10))])

## create pipeline for Random Forest Classifier

In [46]:
random_pipe = Pipeline([('imputer', SimpleImputer(strategy='median')),
                ('scaler', RobustScaler()), 
                ('feature_select', SelectKBest(score_func=mutual_info_classif, k=19)),
                ('clf_rand', RandomForestClassifier(criterion='log_loss',
                                                   max_depth=20, min_samples_split=10))])

## calculate initial cross-validation scores

In [39]:
grad_initial_score = cross_val_score(gradient_pipe, X_train, y_train, cv=5).mean()
rand_initial_score = cross_val_score(random_pipe, X_train, y_train, cv=5).mean()

In [40]:
display(f"The initial score for the Gradient Boosting Pipeline is {grad_initial_score}.")
display(f"The initial score for the Random Forest Pipeline is {rand_initial_score}.")

'The initial score for the Gradient Boosting Pipeline is 0.8574257425742575.'

'The initial score for the Random Forest Pipeline is 0.8831683168316833.'

# Grid-Search to find the best parameters for each pipeline

## Grid-Search for Gradient Boosting Classifier

In [47]:
param_grid = {'imputer__strategy':['mean', 'median', 'most_frequent'],
             'clf_grad__learning_rate': [0.1, 0.15, 0.2, 0.3],
             'clf_grad__loss': ['log_loss', 'exponential'],
             'clf_grad__max_depth': [3, 10, 20, 50, 100],
              'clf_grad__min_samples_leaf': [1, 3, 5, 8, 10]}



In [48]:
search = GridSearchCV(gradient_pipe, param_grid=param_grid, cv=5, n_jobs=-1)

In [49]:
search.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                       ('scaler', RobustScaler()),
                                       ('feature_select',
                                        SelectKBest(k=19,
                                                    score_func=<function mutual_info_classif at 0x7fc19f7c5ee0>)),
                                       ('clf_grad',
                                        GradientBoostingClassifier(max_depth=10,
                                                                   min_samples_leaf=5))]),
             n_jobs=-1,
             param_grid={'clf_grad__learning_rate': [0.1, 0.15, 0.2, 0.3],
                         'clf_grad__loss': ['log_loss', 'exponential'],
                         'clf_grad__max_depth': [3, 10, 20, 50, 100],
                         'clf_grad__min_samples_leaf': [1, 3, 5, 8, 10],
                         'imputer__strategy': ['mean', 'median',
                                               'most_frequent']})

In [50]:
search.best_params_

{'clf_grad__learning_rate': 0.3,
 'clf_grad__loss': 'log_loss',
 'clf_grad__max_depth': 20,
 'clf_grad__min_samples_leaf': 10,
 'imputer__strategy': 'most_frequent'}

In [51]:
search.best_score_

0.8792079207920793

At this stage, the best parameters for the Gradient Boosting Classifier are:

> 'learning_rate': 0.3,

> 'loss': 'log_loss',
 
> 'max_depth': 20,
 
> 'min_samples_leaf': 10

> 'strategy': 'most_frequent'

The best score is:
> 0.8792079207920793


## Grid-Search for Random Forest Classifier

In [53]:
param_grid_rand = {'imputer__strategy':['mean', 'median', 'most_frequent'],
             'clf_rand__criterion': ['gini', 'entropy', 'log_loss'],
             'clf_rand__max_depth': [3, 10, 20, 50, 100],
              'clf_rand__min_samples_split': [2, 3, 5, 8, 10]}

In [54]:
search_rand = GridSearchCV(random_pipe, param_grid=param_grid_rand, cv=5, n_jobs=-1)

In [55]:
search_rand.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('imputer',
                                        SimpleImputer(strategy='median')),
                                       ('scaler', RobustScaler()),
                                       ('feature_select',
                                        SelectKBest(k=19,
                                                    score_func=<function mutual_info_classif at 0x7fc19f7c5ee0>)),
                                       ('clf_rand',
                                        RandomForestClassifier(criterion='log_loss',
                                                               max_depth=20,
                                                               min_samples_split=10))]),
             n_jobs=-1,
             param_grid={'clf_rand__criterion': ['gini', 'entropy', 'log_loss'],
                         'clf_rand__max_depth': [3, 10, 20, 50, 100],
                         'clf_rand__min_samples_split': [2, 3, 5, 8, 10],
                         'imputer__strategy': ['mean', 'median',
                                               'most_frequent']})

In [56]:
search_rand.best_params_

{'clf_rand__criterion': 'gini',
 'clf_rand__max_depth': 20,
 'clf_rand__min_samples_split': 5,
 'imputer__strategy': 'most_frequent'}

In [57]:
search_rand.best_score_

0.8930693069306932

At this stage, the best parameters for the Random Forest Classifier are:
> 'criterion: 'log_loss',

> 'max_depth: 20,

> 'min_samples_split': 10,

> 'strategy': 'median'

The best score is:

> 0.895049504950495


# Random Forest to predict Test Set

In [58]:
random_pipe.fit(X_train, y_train)
predict = random_pipe.score(X_test, y_test)

In [59]:
predict

0.9133858267716536

# Random Forest to predict unseen data

In [60]:
predict_unseen = random_pipe.predict(new_X.drop(columns=['date', 'Value_classification']))

In [61]:
predict_unseen

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [62]:
df = pd.DataFrame(new_X['date'])

In [63]:
df['prediction'] = predict_unseen

In [64]:
df['current_price'] = new_X['current_price']

In [65]:
df

,date,prediction,current_price
632,2022-05-13,0,8.769325
633,2022-05-14,0,10.487976
634,2022-05-15,0,11.376092
635,2022-05-16,0,11.800641
636,2022-05-17,0,10.661735
637,2022-05-18,0,11.053819
638,2022-05-19,0,9.517212
639,2022-05-20,0,10.071635
640,2022-05-21,0,9.695758
641,2022-05-22,0,9.996885
